In [ ]:
# Script to extract population categories for specific storm reports

In [ ]:
# Import necessary modules
import numpy as np
from scipy.spatial import KDTree
import pandas as pd
import xarray as xr
import geopandas as gpd
import geojson as gj
from shapely.geometry import shape
import time

In [ ]:
# Read in the population dataset
pop_data = xr.open_dataset('/chinook2/nathane1/research/landuse/landscan2017.nc')
pop_data

In [ ]:
# Clip the population dataset to allow for stacking of lat/lon arrays

geometry = '''{"type": "polygon",
                    "coordinates": [
                    [
                    [-125, 0],
                    [-70, 0],
                    [-70, 55],
                    [-125, 55],
                    [-125, 0]
                    ]
                ]
                }'''
min_x, min_y, max_x, max_y = shape(gj.loads(geometry)).bounds
meso = pop_data.where((pop_data.lon<=max_x) & (pop_data.lon>=min_x) & (pop_data.lat<=max_y) & (pop_data.lat>=min_y), drop=True)
meso

In [ ]:
# Convert the population dataset to a dataframe for easier lookups 

pop_frame = meso.to_dataframe()
pop_frame = pop_frame.reset_index()
pop_frame

In [ ]:
# Stack arrays to build kd-tree

meso_lat = meso.lat
meso_lon = meso.lon
mesh_lat,mesh_lon = np.meshgrid(meso_lat,meso_lon)
mesh_grid = np.dstack((mesh_lat,mesh_lon))
mesh_grid.shape

In [ ]:
# Reshape grid into two dimensional arrays

grid_two = mesh_grid.reshape(43560000,2)
grid_two

In [ ]:
# Build a kd-tree (WARNING: Very long runtime, only run this once per session)
mytree = KDTree(grid_two)

In [ ]:
# Read in storm report csv; tack on a new column for writing population data
storm_csv = pd.read_csv('/home/nathane1/research/EnvironmentalData/MSRs/2019MSRs.csv')
storm_frame = pd.DataFrame(data = storm_csv)
storm_set = xr.Dataset.from_dataframe(storm_frame)
storm_csv['avg_pop'] = ''
storm_csv

In [ ]:
# Stack storm file arrays
storm_lat = storm_set['location_1_lat'] #Change these back and forth between estimate and measured sets as needed
storm_lon = storm_set['location_1_lon']
storm_grid = np.dstack([storm_lat,storm_lon])[0]

In [ ]:
# Initialize variables for use in the following function

step = 0.0083333334 # <- Initialize the step to be used to get a spatially-averaged population
pop_list = [] # <- Set up an empty list for appending average population values

In [ ]:
#Define a function to match points to the kd-tree; test that function

def run_kdtree(storm_grid):
    distance, indexes = mytree.query(storm_grid) # <- Specifies the distance from the reference point and the index number of that point
    match_lat = grid_two[indexes][0]
    match_lon = grid_two[indexes][1] # <- These two lines pair the first and second elements, respectively, of the index point, to new variables
    min_lat = match_lat - 2 * step
    max_lat = match_lat + 2 * step
    min_lon = match_lon - 2 * step
    max_lon = match_lon + 2 * step # <- These lines specify the range of latitudes and longitudes for the 5x5 grid
    pop_index = pop_frame.loc[(pop_frame['lat'].between(min_lat,max_lat)) & (pop_frame['lon'].between(min_lon,max_lon))].index # <- Find the instances in the dataframe within this range of matching latitudes
    pop_cat = pop_frame['Band1'][pop_index] # <- Find the corresponding average population over the grid 
    average_pop = np.average(pop_cat) # <- Take the average of the population across each grid cell
    pop_list.append(average_pop) # <- Append the average population to the list defined above
    #print('Average population in 5 x 5 km grid is:', average_pop)
    return distance, indexes, pop_cat, pop_index, average_pop # Return various interest variables

In [ ]:
# Iteratively run the kd-tree function to find nearest neighbors for storm reports; append the land use category of the nearest lat/lon pair

start_time = time.time()

for event in storm_grid:
    run_kdtree(event)
storm_csv['avg_pop'] = pop_list
storm_csv.to_csv('/home/nathane1/research/EnvironmentalData/MSRs_with_pop/2019MSR_WithPop.csv')
print('Program complete!')
print('Took --- %s seconds ---' %(time.time() - start_time))